# Proyecto CBIR Milan Campos Alejandro Salvador

### Notebook para la generación de vector Fais y extracción de características con los 2 métodos implementados.

## Instalación de dependencias e imports necesarios

In [16]:
# ---------------------------------------------
# Paso 0: Instalar librerías faltantes
# ---------------------------------------------

!pip install faiss-cpu  # Instalamos FAISS con soporte para GPU
!pip install tensorflow==2.9.2  # Aseguramos compatibilidad con Keras
!pip install keras==2.9.0
!pip install scikit-image  # Para skimage
!pip install rarfile  # Si no lo instalaste antes
!pip install PyWavelets
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.1 MB/s eta 0:00:00


In [39]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import joblib
import faiss
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from skimage.feature import local_binary_pattern, hog
from sklearn.cluster import MiniBatchKMeans
import pandas as pd
import warnings
import logging
from sklearn.decomposition import PCA
import pywt
import umap

In [40]:
# Ignorar advertencias de PIL
warnings.filterwarnings("ignore", category=UserWarning, module="PIL")

# ---------------------------------------------
# Paso 1: Montar Google Drive
# ---------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# Definir la ruta a la carpeta de índices
DB_PATH = '/content/indices'
os.makedirs(DB_PATH, exist_ok=True)

# Definir la carpeta de Google Drive para respaldos
drive_folder = '/content/drive/MyDrive/indices'
os.makedirs(drive_folder, exist_ok=True)

print("Preparación inicial completada.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Preparación inicial completada.


### Importar la base de datos de las imágenes para entrenamiento

In [41]:
# ---------------------------------------------
# Bloque 2: Extraer el Archivo .rar
# ---------------------------------------------

# Ruta al archivo .rar en Google Drive
rar_path = '/content/drive/MyDrive/ING/CBIR/TRAIN.rar'  # Ajusta esta ruta si es necesario

# Ruta donde deseas extraer los archivos
extract_path = '/content/database'  # Puedes cambiar esta ruta si lo deseas

# Asegurarse de que la carpeta de extracción existe
os.makedirs(extract_path, exist_ok=True)

# Instalar 'unrar' si no está instalado
!apt-get install unrar -y

# Extraer el archivo .rar
!unrar x -y "{rar_path}" "{extract_path}/"

print(f'Archivos extraídos en {extract_path}')


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/ING/CBIR/TRAIN.rar

Extracting  /content/database/TRAIN/Bosque/11640.jpg                       0%  OK 
Extracting  /content/database/TRAIN/Bosque/11656.jpg                       1%  OK 
Extracting  /content/database/TRAIN/Bosque/12126.jpg                       2%  OK 
Extracting  /content/database/TRAIN/Bosque/12346.jpg                       2%  OK 
Extracting  /content/database/TRAIN/Bosque/12474.jpg                       3%  OK 
Extracting  /content/database/TRAIN/Bosque/13386.jpg                       4%  OK 
Extracting  /content/database/TRAIN/Bosque/13503.jpg                       5%  OK 
Extracting 

## Función para la reducción de ruido de las imágenes

In [79]:
def reducir_ruido_handcrafted(image):
    # Convertir la imagen a escala de grises (si no lo está ya)
    if len(image.shape) == 3:  # Si la imagen tiene 3 canales (BGR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar un filtro bilateral para preservar bordes
    denoised_image = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

    # Convertir la imagen a color (RGB) si está en escala de grises
    if len(denoised_image.shape) == 2:  # Si la imagen tiene un solo canal (grayscale)
        denoised_image = cv2.cvtColor(denoised_image, cv2.COLOR_GRAY2BGR)

    return denoised_image

## Desarrollo de las clases de extracción de características

In [85]:
# ---------------------------------------------
# Paso 3: Definir Clases y Funciones de Extracción de Características
# ---------------------------------------------

class Handcrafted:
    def __init__(self, imagen, target_size=(150, 150)):
        imagen0 = cv2.imread(imagen)
        imagen = reducir_ruido_handcrafted(imagen0)
        if imagen is None:
            raise ValueError(f"No se pudo leer la imagen {imagen}")
        if imagen.shape[1] != target_size[0] or imagen.shape[0] != target_size[1]:
            self.imagen_color = cv2.resize(imagen, target_size)
        else:
            self.imagen_color = imagen
        self.imagen_grayscale = cv2.cvtColor(self.imagen_color, cv2.COLOR_BGR2GRAY)

        # Inicialización de vectores de características
        self.vector_color = []
        self.vector_textura = []
        self.vector_forma = []
        self.vector_hog = []
        self.vector_histograma_color = []
        self.vector_sobel = []
        self.vector_fft = []
        self.vector_hu = []
        self.vector_wavelet = []

    def extraccion_color_dcd(self, num_colores_dominantes=8, max_samples=10000):
      imagen_norm = self.imagen_color.astype(np.float32) / 255.0
      # Asegurar que los píxeles están en el rango [0, 1]
      pixels = imagen_norm.reshape(-1, 3)

      # Reducir la cantidad de píxeles usados en K-Means mediante muestreo
      if len(pixels) > max_samples:
          indices = np.random.choice(len(pixels), max_samples, replace=False)
          pixels_sample = pixels[indices]
      else:
          pixels_sample = pixels

      # Aplicar K-Means para encontrar colores dominantes
      kmeans = MiniBatchKMeans(n_clusters=num_colores_dominantes, n_init=12, batch_size=1000)
      labels = kmeans.fit_predict(pixels_sample)
      colores_dominantes = kmeans.cluster_centers_

      # Calcular las frecuencias de cada clúster
      frecuencias = np.bincount(labels, minlength=num_colores_dominantes)
      frecuencias_normalizadas = frecuencias / frecuencias.sum()

      # Concatenar los colores dominantes y sus frecuencias
      self.vector_color = np.concatenate([colores_dominantes.flatten(), frecuencias_normalizadas]).astype(np.float32)

      print(f"Vector Color DCD: {len(self.vector_color)}")

    def extraccion_textura_ldrp(self, radio=3, puntos=40, metodo='uniform'):
        ldrp = local_binary_pattern(self.imagen_grayscale, puntos, radio, method=metodo)
        hist, _ = np.histogram(ldrp.ravel(), bins=np.arange(0, puntos + 3), range=(0, puntos + 2))
        self.vector_textura = (hist / hist.sum()).astype(np.float32)  # Normalización opcional global
        print(f"Vector Textura LDRP: {len(self.vector_textura)}")

    def extraccion_forma(self, max_contornos=16):
        _, binary_image = cv2.threshold(self.imagen_grayscale, 128, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        forma_features = []

        for contour in contours[:max_contornos]:
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h if h != 0 else 0
            circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter != 0 else 0
            forma_features.extend([area, perimeter, aspect_ratio, circularity])

        while len(forma_features) < max_contornos * 4:
            forma_features.extend([0, 0, 0, 0])

        self.vector_forma = np.array(forma_features, dtype=np.float32)
        print(f"Vector Forma: {len(self.vector_forma)}")

    def extraccion_caracteristicas_HOG(self, orientaciones=6, pixeles_por_celda=(28, 28), celdas_por_bloque=(1, 1)):
        imagen_norm = self.imagen_grayscale.astype(np.float32) / 255.0
        fd = hog(
            imagen_norm,
            orientations=orientaciones,
            pixels_per_cell=pixeles_por_celda,
            cells_per_block=celdas_por_bloque,
            feature_vector=True
        )
        self.vector_hog = np.array(fd, dtype=np.float32)
        print(f"Vector HOG: {len(self.vector_hog)}")

    def extraccion_histograma_color(self, bins=64):
        # Asegurar que la imagen está en formato uint8
        imagen_color_uint8 = self.imagen_color
        if imagen_color_uint8.dtype != np.uint8:
            imagen_color_uint8 = (imagen_color_uint8 * 255).astype(np.uint8)

        histograma = []
        for i in range(3):  # Canales B, G, R
            hist = cv2.calcHist([imagen_color_uint8], [i], None, [bins], [0, 256])
            hist = hist.flatten()
            histograma.extend(hist)
        self.vector_histograma_color = np.array(histograma, dtype=np.float32)

        print(f"Vector Histograma Color: {len(self.vector_histograma_color)}")

    def extraccion_sobel(self, bins=64):
        imagen_norm = self.imagen_grayscale.astype(np.float32) / 255.0
        sobelx = cv2.Sobel(imagen_norm, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(imagen_norm, cv2.CV_64F, 0, 1, ksize=3)
        magnitude = np.sqrt(sobelx**2 + sobely**2)
        hist, _ = np.histogram(magnitude, bins=bins)
        self.vector_sobel = hist.astype(np.float32)
        print(f"Vector Sobel Histogram: {len(self.vector_sobel)}")

    def extraccion_fft(self, bins=64):
        imagen_norm = self.imagen_grayscale.astype(np.float32) / 255.0
        f_transform = np.fft.fft2(imagen_norm)
        f_shift = np.fft.fftshift(f_transform)
        magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1e-7)
        hist, _ = np.histogram(magnitude_spectrum, bins=bins)
        self.vector_fft = hist.astype(np.float32)
        print(f"Vector FFT Histogram: {len(self.vector_fft)}")

    def extraccion_hu_moments(self):
        moments = cv2.moments(self.imagen_grayscale)
        hu_moments = cv2.HuMoments(moments).flatten()

        # Normalizar los Momentos de Hu aplicando la transformación logarítmica
        epsilon = 1e-7  # Pequeño valor para evitar log(0)
        hu_moments_log = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + epsilon)

        # Guardar el resultado normalizado
        self.vector_hu = hu_moments_log.astype(np.float32)
        print(f"Vector Hu Moments (Normalizados): {len(self.vector_hu)}")

    def extraccion_wavelet(self, wavelet='db1', niveles=3):
        coeffs = pywt.wavedec2(self.imagen_grayscale, wavelet=wavelet, level=niveles)
        wavelet_features = []
        for level in coeffs[1:]:
            cH, cV, cD = level
            wavelet_features.extend([
                cH.mean(), cH.std(),
                cV.mean(), cV.std(),
                cD.mean(), cD.std()
            ])
        self.vector_wavelet = np.array(wavelet_features, dtype=np.float32)
        print(f"Vector Wavelet: {len(self.vector_wavelet)}")


    def concatenar_caracteristicas(self):
        """
        Concatenar todas las características extraídas.
        """
        concatenated_vector = np.concatenate((
            self.vector_color,
            self.vector_textura,
            self.vector_forma,
            self.vector_hog,
            self.vector_histograma_color,
            self.vector_sobel,
            self.vector_fft,
            self.vector_hu,
            self.vector_wavelet,
        ))
        print('largo_vector_concatenado', len(concatenated_vector))

        #print(f"Tamaño del vector concatenado: {len(concatenated_vector)}")
        return concatenated_vector


In [86]:
# Cargar el modelo VGG16 con la capa conv5_3 como salida
def load_VGG16_model():
    base_model = VGG16(weights='imagenet')
    # Modificar el modelo para que la salida sea la capa conv5_3
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_conv3').output)
    return model

# Función para extraer características de la capa conv5_3
def extract_features_VGG16(image_path, model):
    # Cargar y preprocesar la imagen
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Obtener las características de la capa conv5_3
    features = model.predict(img_array)

    # Aplanar las características para convertirlas en un vector
    features_flattened = features.flatten()

    return features_flattened

def normalizar_l2(vector):
    norm = np.linalg.norm(vector)
    if norm == 0:
        return vector
    return vector / norm

## Recuperacion de imagenes y clases




In [87]:
# ---------------------------------------------
# Bloque 5: Recopilación de Rutas de Imágenes y Etiquetas y Creación de CSV
# ---------------------------------------------

# Ruta a la base de datos de imágenes
database_path = '/content/database'  # Ajusta según sea necesario

# Listas para rutas y etiquetas
all_image_paths = []
all_labels = []

# Recorrer todas las carpetas y subcarpetas para recopilar rutas de imágenes y etiquetas
for root, dirs, files in os.walk(database_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(root, file)
            all_image_paths.append(img_path)
            # Extraer la etiqueta (nombre de la carpeta que contiene la imagen)
            label = os.path.basename(os.path.dirname(img_path))
            all_labels.append(label)

print(f"Total de imágenes encontradas: {len(all_image_paths)}")

# Crear un DataFrame y guardarlo como CSV
df = pd.DataFrame({
    'image_path': all_image_paths,
    'label': all_labels
})

csv_path = os.path.join(DB_PATH, 'images_labels.csv')
df.to_csv(csv_path, index=False)
print(f"Archivo CSV creado en {csv_path}")


Total de imágenes encontradas: 180
Archivo CSV creado en /content/indices/images_labels.csv


## Extracción de Características y Generación de Índices FAISS

In [94]:
# ---------------------------------------------
# Bloque 6: Extracción de Características y Generación de Índices FAISS
# ---------------------------------------------
# Inicializar listas para almacenar características
caracteristicas_handcrafted = []
caracteristicas_vgg16 = []

# Cargar el modelo VGG16 una sola vez
vgg_model = load_VGG16_model()

# Extraer características y almacenar rutas y etiquetas en el DataFrame
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    img_path = row['image_path']
    label = row['label']
    try:
        # Extraer características Handcrafted
        extraccion = Handcrafted(img_path)
        extraccion.extraccion_color_dcd()
        extraccion.extraccion_textura_ldrp()
        extraccion.extraccion_forma()
        extraccion.extraccion_caracteristicas_HOG()
        extraccion.extraccion_histograma_color()
        extraccion.extraccion_sobel()
        extraccion.extraccion_fft()
        extraccion.extraccion_hu_moments()
        extraccion.extraccion_wavelet()
        vector_handcrafted = extraccion.concatenar_caracteristicas()
        caracteristicas_handcrafted.append(vector_handcrafted)

        # Extraer características con VGG16 (CNN)
        vector_vgg16 = extract_features_VGG16(img_path, vgg_model)
        vector_vgg16_norm = normalizar_l2(vector_vgg16)
        caracteristicas_vgg16.append(vector_vgg16_norm)

    except Exception as e:
        logging.error(f"Error procesando {img_path}: {e}")

# Convertir las listas de características a matrices NumPy
caracteristicas_handcrafted_np = np.array(caracteristicas_handcrafted, dtype='float32')
caracteristicas_vgg16_np = np.array(caracteristicas_vgg16, dtype='float32')

logging.info(f"Se han extraído {len(caracteristicas_handcrafted_np)} vectores Handcrafted y {len(caracteristicas_vgg16_np)} vectores VGG16.")



  0%|          | 0/180 [00:00<?, ?it/s]

Vector Color DCD: 32
Vector Textura LDRP: 42
Vector Forma: 64
Vector HOG: 150
Vector Histograma Color: 192
Vector Sobel Histogram: 64
Vector FFT Histogram: 64
Vector Hu Moments (Normalizados): 7
Vector Wavelet: 18
largo_vector_concatenado 633
1/1 [==============================] - 1s 998ms/step


  1%|          | 1/180 [00:01<03:31,  1.18s/it]

Vector Color DCD: 32
Vector Textura LDRP: 42
Vector Forma: 64
Vector HOG: 150
Vector Histograma Color: 192
Vector Sobel Histogram: 64
Vector FFT Histogram: 64
Vector Hu Moments (Normalizados): 7
Vector Wavelet: 18
largo_vector_concatenado 633
1/1 [==============================] - 1s 885ms/step


  1%|          | 2/180 [00:02<03:18,  1.12s/it]

...

 99%|█████████▉| 178/180 [02:11<00:01,  1.45it/s]

Vector Color DCD: 32
Vector Textura LDRP: 42
Vector Forma: 64
Vector HOG: 150
Vector Histograma Color: 192
Vector Sobel Histogram: 64
Vector FFT Histogram: 64
Vector Hu Moments (Normalizados): 7
Vector Wavelet: 18
largo_vector_concatenado 633
1/1 [==============================] - 0s 490ms/step


 99%|█████████▉| 179/180 [02:12<00:00,  1.48it/s]

Vector Color DCD: 32
Vector Textura LDRP: 42
Vector Forma: 64
Vector HOG: 150
Vector Histograma Color: 192
Vector Sobel Histogram: 64
Vector FFT Histogram: 64
Vector Hu Moments (Normalizados): 7
Vector Wavelet: 18
largo_vector_concatenado 633
1/1 [==============================] - 0s 484ms/step


100%|██████████| 180/180 [02:12<00:00,  1.35it/s]


## Normalización Global de Características Handcrafted

In [100]:
# ---------------------------------------------
# Bloque 7: Normalización Global de Características Handcrafted
# ---------------------------------------------

# Seleccionar la técnica de normalización
scaler_handcrafted = StandardScaler()

# Ajustar el escalador en todas las características Handcrafted
scaler_handcrafted.fit(caracteristicas_handcrafted_np)

# Transformar las características Handcrafted
caracteristicas_handcrafted_normalizadas = scaler_handcrafted.transform(caracteristicas_handcrafted_np)

# Guardar el escalador para uso futuro
scaler_path = os.path.join(DB_PATH, 'scaler_handcrafted.pkl')
joblib.dump(scaler_handcrafted, scaler_path)
print(f"Escalador Handcrafted guardado en {scaler_path}")

# Guardar las características Handcrafted normalizadas
caracteristicas_normalizadas_path = os.path.join(DB_PATH, 'caracteristicas_handcrafted_normalizadas.npy')
np.save(caracteristicas_normalizadas_path, caracteristicas_handcrafted_normalizadas)
print(f"Características Handcrafted normalizadas guardadas en {caracteristicas_normalizadas_path}")

# Verificar las estadísticas después de la normalización
print(f"Características Handcrafted normalizadas - min: {caracteristicas_handcrafted_normalizadas.min()}, "
      f"max: {caracteristicas_handcrafted_normalizadas.max()}, "
      f"mean: {caracteristicas_handcrafted_normalizadas.mean()}, "
      f"std: {caracteristicas_handcrafted_normalizadas.std()}")


Escalador Handcrafted guardado en /content/indices/scaler_handcrafted.pkl
Características Handcrafted normalizadas guardadas en /content/indices/caracteristicas_handcrafted_normalizadas.npy
Características Handcrafted normalizadas - min: -12.277078628540039, max: 13.379088401794434, mean: -1.3391951436680927e-10, std: 0.9976274967193604


## Crear y guardar Índices Faiss

In [101]:
# ---------------------------------------------
# Bloque 7: crear y guardar indices
# ---------------------------------------------

# Crear índice FAISS para Handcrafted
d_handcrafted = caracteristicas_handcrafted_normalizadas.shape[1]
index_handcrafted = faiss.IndexFlatL2(d_handcrafted)
index_handcrafted.add(caracteristicas_handcrafted_normalizadas)

# Verificar que las características se han añadido correctamente
print(f"Índice Handcrafted - número de vectores añadidos: {index_handcrafted.ntotal}")

# Guardar el índice FAISS de Handcrafted
index_handcrafted_path = os.path.join(DB_PATH, 'index_handcrafted.index')
faiss.write_index(index_handcrafted, index_handcrafted_path)
print(f"Índice FAISS Handcrafted guardado en {index_handcrafted_path}")

# Crear índice FAISS para VGG16 (ya normalizados con L2)
d_vgg16 = caracteristicas_vgg16_np.shape[1]
index_vgg16 = faiss.IndexFlatL2(d_vgg16)
index_vgg16.add(caracteristicas_vgg16_np)

# Verificar que las características se han añadido correctamente
print(f"Índice VGG16 - número de vectores añadidos: {index_vgg16.ntotal}")

# Guardar el índice FAISS de VGG16
index_vgg16_path = os.path.join(DB_PATH, 'index_vgg16.index')
faiss.write_index(index_vgg16, index_vgg16_path)
print(f"Índice FAISS VGG16 guardado en {index_vgg16_path}")

print(f"Dimensión de los vectores de características: {caracteristicas_handcrafted_np.shape}")


Índice Handcrafted - número de vectores añadidos: 180
Índice FAISS Handcrafted guardado en /content/indices/index_handcrafted.index
Índice VGG16 - número de vectores añadidos: 180
Índice FAISS VGG16 guardado en /content/indices/index_vgg16.index
Dimensión de los vectores de características: (180, 633)


## Guardar Etiquetas y rutas

In [102]:
# ---------------------------------------------
# Bloque 10: Crear Archivo CSV con Nombre de Imagen y Label
# ---------------------------------------------

# Crear un DataFrame con nombre de imagen y etiqueta
df_csv = pd.DataFrame({
    'image_name': [os.path.basename(path) for path in all_image_paths],
    'label': all_labels
})

# Definir la ruta para guardar el CSV
csv_path = os.path.join(DB_PATH, 'images_labels.csv')

# Guardar el DataFrame como CSV
df_csv.to_csv(csv_path, index=False)
print(f"Archivo CSV creado en {csv_path}")


Archivo CSV creado en /content/indices/images_labels.csv


In [103]:
# ---------------------------------------------
# Bloque 11: Copiar Índices y Datos a Google Drive
# ---------------------------------------------

# Lista de archivos a copiar
archivos_a_copiar = [
    'index_handcrafted.index',
    'index_vgg16.index',
    'scaler_handcrafted.pkl',
    'images_labels.csv'
]

# Copiar cada archivo a Google Drive
for archivo in archivos_a_copiar:
    src = os.path.join(DB_PATH, archivo)
    dst = os.path.join(drive_folder, archivo)
    if os.path.exists(src):
        !cp "{src}" "{dst}"
        print(f"Archivo copiado a Google Drive: {archivo}")
    else:
        print(f"El archivo no existe y no se pudo copiar: {archivo}")


Archivo copiado a Google Drive: index_handcrafted.index
Archivo copiado a Google Drive: index_vgg16.index
Archivo copiado a Google Drive: scaler_handcrafted.pkl
Archivo copiado a Google Drive: images_labels.csv


In [104]:
# Información detallada del DataFrame
print("\nInformación del DataFrame:")
print(df.info())

# Descripción estadística de las etiquetas
print("\nDistribución de etiquetas:")
print(df['label'].value_counts())



Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_path  180 non-null    object
 1   label       180 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB
None

Distribución de etiquetas:
label
Montaña      30
Oceano       30
Bosque       30
Glaciares    30
Edificios    30
Calles       30
Name: count, dtype: int64
